In [ ]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate
!apt install git-lfs

In [2]:
import torch
from torch.nn import CrossEntropyLoss
from torch.utils.data.dataloader import DataLoader
from torch.optim import AdamW
from torch.utils.tensorboard import SummaryWriter
%load_ext tensorboard

from huggingface_hub import notebook_login, Repository
from datasets import load_dataset, DatasetDict, Dataset
from accelerate import Accelerator
from transformers import AutoTokenizer, GPT2LMHeadModel, AutoConfig, pipeline, get_constant_schedule_with_warmup, get_cosine_schedule_with_warmup, get_linear_schedule_with_warmup
from transformers.keras_callbacks import PushToHubCallback

import re
import json
import os
from tqdm import tqdm
from tqdm.notebook import tqdm
from google.colab import drive

### TRAINING


#### HYPER PARAMS, GLOBAL VARS, FUNCTIONS

In [3]:
# HYPER PARAMETERS
context_length                 = 1024
train_batch_size               = 4
eval_batch_size                = 8
weight_decay                   = 0.1
lr                             = 5e-4
lr_scheduler_func              = get_cosine_schedule_with_warmup
adamw_b1                       = 0.9
adamw_b2                       = 0.95
adamw_e                        = 1e-8
num_warmup_steps               = 15
gradient_accumulation_steps    = 8
gradient_checkpointing         = False
eval_steps                     = 100
num_train_epochs               = 8
mixed_precision                = "fp16"


#GLOBAL VARS
have_git_write_access          = True                              # set to True if you need to commit changes
user_email                     = "orest.andrusyshyn@ucu.edu.ua"    # only needed if above have_git_write_access == True
user_name                      = "Orest Andrusyshyn"               # only needed if above have_git_write_access == True

drive_mounted                  = True
drive_mounted_path             = '/content/gdrive/'    # only needed if drive_mounted == True
save_json_logs                 = True                  # set to True to save training results into json file (drive_train_res_path)
drive_train_res_path           = drive_mounted_path + 'My Drive/UCU/diploma/training_results/v30_a95_lrcos5em4.json'    # only needed if save_json_logs == True
save_tensorboard_logs          = True                  # set to True to save training results into tensorboard run (tensorboard_run_path)
tensorboard_run_path           = drive_mounted_path + 'My Drive/UCU/diploma/tensorboard_runs/v30_a95_lrcos5em4'                   # only needed if save_tensorboard_logs == True

data_archived                  = True    # set to True if you need to unarchive the data
raw_train_archive              = drive_mounted_path + "My Drive/UCU/diploma/datasets/dataset_train.zip"         # only needed if data_archived == True
raw_valid_archive              = drive_mounted_path + "My Drive/UCU/diploma/datasets/dataset_valid.zip"    # only needed if data_archived == True
raw_train_json                 = "./dataset_train.json"       # if data_archived == True, json file will be in the current working directory
raw_valid_json                 = "./dataset_valid.json"       # if data_archived == True, json file will be in the current working directory

tokenizer_repo_name            = "Andrusyshyn/gpt2-coq-tokenizer"
tokenizer_commit_hash          = "4e04ba257c544aaad0de61a2d82e64ba22e70847"

init_model                     = "gpt2"                                                   # base model Hugging Face name
model_repo_name                = "Andrusyshyn/gpt2-pretrained-for-coq-pt-custom-train"
model_output_dir               = "./gpt2-pretrained-for-coq-pt-custom-train-local"        # local dir to save the model

push_to_hub                    = True                 # set to True to push the model
run_name                       = "v30_a95_lrcos5em4"    # branch name (only needed if push_to_hub == True)

partially_trained              = True                                         # set to True to continue model training from specific commit hash (model_commit_hash)
model_commit_hash              = "bed04d11daf0a71769a16b61546005f69a1d4dd4"    # only needed if partially_trained == True
previously_completed_steps     = 2917                                          # only needed if partially_trained == True
previous_global_steps          = 23336                                         # only needed if partially_trained == True
previous_step                  = 23334                                          # only needed if partially_trained == True
stopped_epoch                  = 4                                             # only needed if partially_trained == True

torch_seed                     = 7
data_seed                      = 23
data_generator = torch.Generator().manual_seed(data_seed)
torch.manual_seed(torch_seed)

In [4]:
# CUSTOM LOSS FUNCTION
def custom_loss(inputs, logits):
    # inputs [batch_size X cl]
    # logits [btach_size X cl X vocab_size]
    # Our labels start from second sequence token because first one does not have preceding token.
    # We drop last logit because last sequence token does not have subsequent token, so no label to compare
    shifted_labels = inputs[..., 1:].contiguous()
    shifted_logits = logits[..., :-1, :].contiguous()

    loss_func = CrossEntropyLoss(reduction='none')
    # loss [batch_size * (cl-1)] = loss_fct([batch_size * (cl-1) X vocab_size], [batch_size * (cl-1)])
    loss = loss_func(shifted_logits.view(-1, shifted_logits.size(-1)), shifted_labels.view(-1))
    # loss_per_sample [batch_size]
    loss_per_sequence = loss.view(shifted_logits.size(0), shifted_logits.size(1)).mean(axis=1)
    return loss_per_sequence.mean()

In [5]:
# PARAMETERS FOR WEIGHT DECAY
def get_wd_parameters(model, no_decay=["bias", r"ln_.{1,2}\.weight"]):
    wd_params = []
    nwd_params = []
    for name, params in model.named_parameters():
        if any(re.search(nd_reg, name) for nd_reg in no_decay):
            nwd_params.append(params)
        else:
            wd_params.append(params)
    return [
        {"params": wd_params,  "weight_decay": weight_decay},
        {"params": nwd_params, "weight_decay": 0.0},
    ]

In [6]:
# TOKENIZE RAW DATASETS
def get_tokenized_dataset(p_raw_dataset, p_context_length, p_tokenizer):
    concatenated_tokenized_samples = []
    for sample in p_raw_dataset:
        tokenized_sample = p_tokenizer(sample["content"], truncation=False)["input_ids"]
        concatenated_tokenized_samples.extend(tokenized_sample + [p_tokenizer.eos_token_id])

    tokenized_dataset_list = []
    for i in range(0, len(concatenated_tokenized_samples), p_context_length):
        input_ids = concatenated_tokenized_samples[i : i + p_context_length]
        if len(input_ids) == p_context_length:
            tokenized_dataset_list.append(torch.tensor(input_ids))

    return Dataset.from_dict({"input_ids": tokenized_dataset_list})

In [7]:
# SAVING TRAINING RESULTS TO THE JSON FILE
def save_results(filepath:str, split: str, results: list):
    """
    {
        "hyperparams": {},
        "train": [
            results1,
            results2,
            ...
        ],
        "valid": [
            results1,
            results2,
            ...
        ]
    }
    """
    if not save_json_logs:
        return
    if split not in {"train", "valid"}:
        print("ERROR: INVALID SPLIT")
        return
    _run_name = run_name
    if not push_to_hub:
        _run_name = ""
    _tensorboard_run_path = tensorboard_run_path
    if not save_tensorboard_logs:
        _tensorboard_run_path = ""
    _lr_scheduler_type = "cosine"
    if lr_scheduler_func == get_linear_schedule_with_warmup:
        _lr_scheduler_type = "linear"
    elif lr_scheduler_func == get_constant_schedule_with_warmup:
        _lr_scheduler_type = "const"
    hyperparams_dict = {
        "context_length"                 : context_length,
        "train_batch_size"               : train_batch_size,
        "eval_batch_size"                : eval_batch_size,
        "weight_decay"                   : weight_decay,
        "lr"                             : lr,
        "lr_scheduler_type"              : _lr_scheduler_type,
        "adamw_b1"                       : adamw_b1,
        "adamw_b2"                       : adamw_b2,
        "adamw_e"                        : adamw_e,
        "num_warmup_steps"               : num_warmup_steps,
        "gradient_accumulation_steps"    : gradient_accumulation_steps,
        "gradient_checkpointing"         : gradient_checkpointing,
        "eval_steps"                     : eval_steps,
        "num_train_epochs"               : num_train_epochs,
        "mixed_precision"                : mixed_precision,
        "tokenizer_repo_name"            : tokenizer_repo_name,
        "tokenizer_commit_hash"          : tokenizer_commit_hash,
        "init_model"                     : init_model,
        "model_repo_name"                : model_repo_name,
        "run_name"                       : _run_name,
        "drive_train_res_path"           : drive_train_res_path,
        "tensorboard_run_path"           : _tensorboard_run_path,
    }
    json_data = {"train": [], "valid": []}
    if os.path.exists(filepath):
        with open(filepath, 'r') as json_file:
            json_data = json.load(json_file)
    json_data["hyperparams"] = hyperparams_dict
    json_data[split].extend(results)
    with open(filepath, 'w') as json_file:
        json.dump(json_data, json_file, indent=4)

In [8]:
# TENSORBOARD VISUALISATION
def add_to_tensorboard(json_filepath, tensorboard_run_path):
    if not save_tensorboard_logs:
        return
    if not os.path.exists(json_filepath):
        print("ERROR: json_filepath DOES NOT EXIST")
        return
    with open(json_filepath, mode="r") as json_file:
        json_data = json.load(json_file)
        one_step_tokens = json_data["hyperparams"]["train_batch_size"] * json_data["hyperparams"]["gradient_accumulation_steps"] * json_data["hyperparams"]["context_length"]
        writer = SummaryWriter(tensorboard_run_path)
        prev_comleted_steps = json_data["train"][0]["completed_steps"]
        prev_lr = json_data["train"][0]["lr"][0]
        train_losses = []
        cs = 0
        for entry in json_data["train"]:
            cs = entry["completed_steps"]
            if cs == prev_comleted_steps:
                train_losses.append(entry["loss/train"])
                continue
            else:
                writer.add_scalar("Loss/Train", sum(train_losses)/len(train_losses), prev_comleted_steps * one_step_tokens)
                writer.add_scalar("Learning Rate", prev_lr, prev_comleted_steps * one_step_tokens)
                train_losses = [entry["loss/train"]]
                prev_comleted_steps = cs
                prev_lr = entry["lr"][0]
        writer.add_scalar("Loss/Train", sum(train_losses)/len(train_losses), cs * one_step_tokens)
        writer.add_scalar("Learning Rate", prev_lr, cs * one_step_tokens)

        for entry in json_data["valid"]:
            cs = entry["completed_steps"]
            writer.add_scalar("Loss/Eval", entry["loss/eval"], cs * one_step_tokens)
            writer.add_scalar("Perplexity/Eval", entry["perplexity"], cs * one_step_tokens)
        writer.close()

#### CONFIGURING

In [9]:
# MOUNTING DRIVE FOR SAVING LOGS
if drive_mounted:
    drive.mount(drive_mounted_path)

Mounted at /content/gdrive/


In [10]:
# CONFIGURING GIT CREDENTIALS
if have_git_write_access:
    !git config --global user.email "{user_email}"
    !git config --global user.name "{user_name}"

# To set Hugging Face token (for writing access) create HF_TOKEN secret in Google Collab or use notebook_login()

In [11]:
# CONFIGURING GIT DIRECTORIES
if push_to_hub:
    repo = Repository(model_output_dir, clone_from=model_repo_name)
    repo.git_checkout(run_name, create_branch_ok=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/Andrusyshyn/gpt2-pretrained-for-coq-pt-custom-train into local empty directory.


Download file model.safetensors:   0%|          | 8.00k/480M [00:00<?, ?B/s]

Clean file model.safetensors:   0%|          | 1.00k/480M [00:00<?, ?B/s]

Checked out v30_a95_lrcos5em4 from v30_a95_lrcos5em4.
Branch 'v30_a95_lrcos5em4' set up to track remote branch 'v30_a95_lrcos5em4' from 'origin'.




#### DATASETS

In [12]:
# UNPACK DATASETS
if data_archived:
    if raw_train_archive.endswith(".gz"):
        !gzip -dkv "{raw_train_archive}"
    if raw_train_archive.endswith(".zip"):
        !unzip "{raw_train_archive}"
    if raw_valid_archive.endswith(".gz"):
        !gzip -dkv "{raw_valid_archive}"
    if raw_valid_archive.endswith(".zip"):
        !unzip "{raw_valid_archive}"

Archive:  /content/gdrive/My Drive/UCU/diploma/datasets/dataset_train.zip
  inflating: dataset_train.json      
Archive:  /content/gdrive/My Drive/UCU/diploma/datasets/dataset_valid.zip
  inflating: dataset_valid.json      


In [13]:
# LOAD DATASETS
data_files = {"train": raw_train_json, "validation": raw_valid_json}
raw_datasets = load_dataset("json", data_files=data_files, field="data")
print(raw_datasets)

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['content', 'filepath'],
        num_rows: 4972
    })
    validation: Dataset({
        features: ['content', 'filepath'],
        num_rows: 742
    })
})


In [14]:
# LOADING TOKENIZER
tokenizer = AutoTokenizer.from_pretrained(tokenizer_repo_name, revision=tokenizer_commit_hash)
print("tokenizer vocab size: ", len(tokenizer))
# TOKENIZE RAW DATASETS
train_dataset = get_tokenized_dataset(raw_datasets["train"],      context_length, tokenizer)
valid_dataset = get_tokenized_dataset(raw_datasets["validation"], context_length, tokenizer)
train_dataset.set_format("torch")
valid_dataset.set_format("torch")

# CREATE DATALOADERS
train_dataloader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True, generator=data_generator)
valid_dataloader = DataLoader(valid_dataset, batch_size=eval_batch_size)
print(train_dataset)
print(valid_dataset)
print("len(train_dataloader): ", len(train_dataloader))
print("len(valid_dataloader): ", len(valid_dataloader))

tokenizer_config.json:   0%|          | 0.00/440 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/465k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/265k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2260 > 1024). Running this sequence through the model will result in indexing errors


tokenizer vocab size:  30000
Dataset({
    features: ['input_ids'],
    num_rows: 18667
})
Dataset({
    features: ['input_ids'],
    num_rows: 3784
})
len(train_dataloader):  4667
len(valid_dataloader):  473


#### CONFIGURING MODEL AND TRAINING

In [15]:
# CONFIGURING MODEL
model = None
if partially_trained:
    print("Loading partially trained model")
    model = GPT2LMHeadModel.from_pretrained(model_repo_name, revision=model_commit_hash)
else:
    print("Training from scratch")
    config = AutoConfig.from_pretrained(
        init_model,
        vocab_size=len(tokenizer),
        n_ctx=context_length,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )
    model = GPT2LMHeadModel(config)
print()
if gradient_checkpointing:
    model.gradient_checkpointing_enable()

optimizer = AdamW(get_wd_parameters(model), lr=lr, betas=(adamw_b1, adamw_b2), eps=adamw_e)
accelerator = Accelerator(mixed_precision=mixed_precision)
model, optimizer, train_dataloader, valid_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, valid_dataloader
)

num_steps_per_epoch = len(train_dataloader)
print("Num steps per epoch: ", num_steps_per_epoch)
num_training_completed_steps = (num_train_epochs * num_steps_per_epoch) // gradient_accumulation_steps
if ((num_train_epochs * num_steps_per_epoch) % gradient_accumulation_steps != 0):
    num_training_completed_steps += 1
print("Num optimizer steps: ", num_training_completed_steps)

if lr_scheduler_func == get_constant_schedule_with_warmup:
    lr_scheduler = lr_scheduler_func(
        optimizer=optimizer,
        num_warmup_steps=num_warmup_steps
    )
else:
    lr_scheduler = lr_scheduler_func(
        optimizer=optimizer,
        num_warmup_steps=num_warmup_steps,
        num_training_steps=num_training_completed_steps
    )


if partially_trained:
    with torch.no_grad():
        for ind in range(previously_completed_steps):
            if ((ind < num_warmup_steps) or (lr_scheduler.get_lr()[0] > (0.1 * lr))):
                lr_scheduler.step()

print()
print(f"Model size:                                        {model.num_parameters()}")
print(f"Model size (only trainable params):                {model.num_parameters(only_trainable=True)}")
print(f"Model size (only trainable non-embeddings params): {model.num_parameters(only_trainable=True, exclude_embeddings=True)}")

Loading partially trained model


config.json:   0%|          | 0.00/946 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]


Num steps per epoch:  4667
Num optimizer steps:  4667

Model size:                                        108882432
Model size (only trainable params):                108882432
Model size (only trainable non-embeddings params): 85056000


In [16]:
# EVALUATION FUNCTION
def evaluate():
    torch.cuda.empty_cache()

    model.eval()
    losses = []
    with torch.no_grad():
        for step, batch in enumerate(valid_dataloader):
            with torch.no_grad():
                logits = model(batch["input_ids"]).logits
                loss = custom_loss(batch["input_ids"], logits)
                losses.append(loss.item())
    loss = torch.mean(torch.Tensor(losses))
    try:
        perplexity = torch.exp(loss)
    except OverflowError:
        perplexity = float("inf")

    torch.cuda.empty_cache()
    return loss.item(), perplexity.item()

In [ ]:
# TRAINING LOOP
log_buffer = []

model.train()
completed_steps = 0
global_steps = 0
if partially_trained:
    completed_steps = previously_completed_steps
    global_steps = previous_global_steps
for epoch in range(num_train_epochs):
    for step, batch in tqdm(
        enumerate(train_dataloader, start=0), total=num_steps_per_epoch
    ):
        if partially_trained and ((epoch<stopped_epoch) or ((epoch==stopped_epoch) and (step <= previous_step))):
            continue

        logits = model(batch["input_ids"]).logits
        loss = custom_loss(batch["input_ids"], logits)
################################################################################
        log_train = {
                "loss/train": loss.item(),
                "completed_steps": completed_steps,
                "lr": lr_scheduler.get_lr(),
                "global_steps" : global_steps,
                "epoch": epoch,
                "steps": step,
        }
        if ((completed_steps % 10 == 0) and (global_steps % gradient_accumulation_steps == 0)):
            accelerator.print(log_train)
        if save_json_logs:
            log_buffer.append(log_train)
################################################################################
        loss = loss / gradient_accumulation_steps
        accelerator.backward(loss)
        global_steps += 1
################################################################################
        if global_steps % gradient_accumulation_steps == 0:
            accelerator.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            if ((completed_steps < num_warmup_steps) or (lr_scheduler.get_lr()[0] > (0.1 * lr))):
                lr_scheduler.step()
            optimizer.zero_grad()
            completed_steps += 1
        if (global_steps % (eval_steps * gradient_accumulation_steps)) == 0:
            if save_json_logs:
                save_results(drive_train_res_path, "train", log_buffer)
                log_buffer = []
            eval_loss, perplexity = evaluate()
            log_eval = {
                    "loss/eval": eval_loss,
                    "perplexity": perplexity,
                    "completed_steps": completed_steps,
                    "lr": lr_scheduler.get_lr(),
                    "global_steps" : global_steps,
                    "epoch": epoch,
                    "steps": step,
                    "loss/train": loss.item() * gradient_accumulation_steps,
            }
            accelerator.print(log_eval)
            save_results(drive_train_res_path, "valid", [log_eval])
            accelerator.wait_for_everyone()
            unwrapped_model = accelerator.unwrap_model(model)
            unwrapped_model.save_pretrained(model_output_dir, save_function=accelerator.save)
            tokenizer.save_pretrained(model_output_dir)
            if accelerator.is_main_process:
                if push_to_hub:
                    repo.push_to_hub(
                        commit_message=f"Training in progress: completed_steps {completed_steps}; global_steps {global_steps};\
                                         epoch {epoch}; steps {step}; lr {lr_scheduler.get_lr()};\
                                         loss/eval {eval_loss}; perplexity {perplexity}; loss/train {loss.item() * gradient_accumulation_steps}",
                        blocking=False
                    )
            model.train()

################################################################################
################################################################################
################################################################################

#GRADIENT UPDATE (In case (global_steps % gradient_accumulation_steps != 0)
last_eval_log_train_loss = 0
if (global_steps % gradient_accumulation_steps != 0):
    for step, batch in tqdm(
        enumerate(train_dataloader, start=0), total=(gradient_accumulation_steps - (global_steps % gradient_accumulation_steps)) - 1    # -1 here is purely for better visualisation of tqdm progress bar
    ):
        logits = model(batch["input_ids"]).logits
        loss = custom_loss(batch["input_ids"], logits)
        last_eval_log_train_loss = loss.item()
        log_train = {
                "loss/train": loss.item(),
                "completed_steps": completed_steps,
                "lr": lr_scheduler.get_lr(),
                "global_steps" : global_steps,
                "epoch": epoch,
                "steps": step + num_steps_per_epoch,
        }
        if save_json_logs:
            log_buffer.append(log_train)
        loss = loss / gradient_accumulation_steps
        accelerator.backward(loss)
        global_steps += 1
        if global_steps % gradient_accumulation_steps == 0:
            accelerator.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            if ((completed_steps < num_warmup_steps) or (lr_scheduler.get_lr()[0] > (0.1 * lr))):
                lr_scheduler.step()
            optimizer.zero_grad()
            completed_steps += 1
            break

################################################################################
################################################################################
################################################################################

# FINAL EVALUATE AND SAVE
additional_steps = 0
if (global_steps % gradient_accumulation_steps != 0):
    additional_steps = gradient_accumulation_steps - (global_steps % gradient_accumulation_steps)
with torch.no_grad():
    last_train_loss = 0
    for batch in train_dataloader:
        logits = model(batch["input_ids"]).logits
        loss = custom_loss(batch["input_ids"], logits)
        last_train_loss = loss.item()
        break
log_train = {
        "loss/train": last_train_loss,
        "completed_steps": completed_steps,
        "lr": lr_scheduler.get_lr(),
        "global_steps" : global_steps,
        "epoch": epoch,
        "steps": num_steps_per_epoch + additional_steps,
}
if save_json_logs:
    log_buffer.append(log_train)
    save_results(drive_train_res_path, "train", log_buffer)
    log_buffer = []
accelerator.print(log_train)

eval_loss, perplexity = evaluate()
log_eval = {
        "loss/eval": eval_loss,
        "perplexity": perplexity,
        "completed_steps": completed_steps,
        "lr": lr_scheduler.get_lr(),
        "global_steps" : global_steps,
        "epoch": epoch,
        "steps": num_steps_per_epoch + additional_steps - 1,
        "loss/train": last_eval_log_train_loss,
}
accelerator.print(log_eval)
save_results(drive_train_res_path, "valid", [log_eval])

accelerator.wait_for_everyone()
unwrapped_model = accelerator.unwrap_model(model)
unwrapped_model.save_pretrained(model_output_dir, save_function=accelerator.save)
tokenizer.save_pretrained(model_output_dir)
if accelerator.is_main_process:
    if push_to_hub:
        repo.push_to_hub(
            commit_message=f"Final model: completed_steps {completed_steps}; global_steps {global_steps};\
                             epoch {epoch}; steps {num_steps_per_epoch + additional_steps - 1}; lr {lr_scheduler.get_lr()};\
                             loss/eval {eval_loss}; perplexity {perplexity}; loss/train {last_eval_log_train_loss}",
            blocking=False
        )

model.train()

torch.cuda.empty_cache()

  0%|          | 0/4667 [00:00<?, ?it/s]

  0%|          | 0/4667 [00:00<?, ?it/s]

  0%|          | 0/4667 [00:00<?, ?it/s]

  0%|          | 0/4667 [00:00<?, ?it/s]

  0%|          | 0/4667 [00:00<?, ?it/s]

...

{'loss/train': 1.4097490310668945, 'completed_steps': 4000, 'lr': [4.991068836893414e-05, 4.991068836893414e-05], 'global_steps': 32000, 'epoch': 6, 'steps': 3997}
{'loss/train': 1.399709701538086, 'completed_steps': 4010, 'lr': [4.991068836893414e-05, 4.991068836893414e-05], 'global_steps': 32080, 'epoch': 6, 'steps': 4077}
{'loss/train': 1.3567140102386475, 'completed_steps': 4020, 'lr': [4.991068836893414e-05, 4.991068836893414e-05], 'global_steps': 32160, 'epoch': 6, 'steps': 4157}
{'loss/train': 1.2883665561676025, 'completed_steps': 4030, 'lr': [4.991068836893414e-05, 4.991068836893414e-05], 'global_steps': 32240, 'epoch': 6, 'steps': 4237}
{'loss/train': 1.278282642364502, 'completed_steps': 4040, 'lr': [4.991068836893414e-05, 4.991068836893414e-05], 'global_steps': 32320, 'epoch': 6, 'steps': 4317}
{'loss/train': 1.1682870388031006, 'completed_steps': 4050, 'lr': [4.991068836893414e-05, 4.991068836893414e-05], 'global_steps': 32400, 'epoch': 6, 'steps': 4397}
{'loss/train': 1.6

  0%|          | 0/4667 [00:00<?, ?it/s]

{'loss/train': 1.2647790908813477, 'completed_steps': 4090, 'lr': [4.991068836893414e-05, 4.991068836893414e-05], 'global_steps': 32720, 'epoch': 7, 'steps': 50}
{'loss/eval': 1.8730721473693848, 'perplexity': 6.508260250091553, 'completed_steps': 4100, 'lr': [4.991068836893414e-05, 4.991068836893414e-05], 'global_steps': 32800, 'epoch': 7, 'steps': 129, 'loss/train': 1.4353737831115723}


Several commits (42) will be pushed upstream.


{'loss/train': 0.7560071349143982, 'completed_steps': 4100, 'lr': [4.991068836893414e-05, 4.991068836893414e-05], 'global_steps': 32800, 'epoch': 7, 'steps': 130}
{'loss/train': 1.2004482746124268, 'completed_steps': 4110, 'lr': [4.991068836893414e-05, 4.991068836893414e-05], 'global_steps': 32880, 'epoch': 7, 'steps': 210}
{'loss/train': 1.0510972738265991, 'completed_steps': 4120, 'lr': [4.991068836893414e-05, 4.991068836893414e-05], 'global_steps': 32960, 'epoch': 7, 'steps': 290}
{'loss/train': 1.1580150127410889, 'completed_steps': 4130, 'lr': [4.991068836893414e-05, 4.991068836893414e-05], 'global_steps': 33040, 'epoch': 7, 'steps': 370}
{'loss/train': 1.21113920211792, 'completed_steps': 4140, 'lr': [4.991068836893414e-05, 4.991068836893414e-05], 'global_steps': 33120, 'epoch': 7, 'steps': 450}
{'loss/train': 1.45094895362854, 'completed_steps': 4150, 'lr': [4.991068836893414e-05, 4.991068836893414e-05], 'global_steps': 33200, 'epoch': 7, 'steps': 530}
{'loss/train': 2.174126148

In [ ]:
torch.cuda.empty_cache()
add_to_tensorboard(drive_train_res_path, tensorboard_run_path)

In [ ]:
%tensorboard --logdir "{tensorboard_run_path}"